In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import rankdata
import seaborn as sns
from sklearn.datasets import load_diabetes, fetch_kddcup99
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, roc_curve,\
                            classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from tqdm import tqdm_notebook as tqdm

In [2]:
%matplotlib notebook

# KDD cup '99 dataset

In [3]:
sf = fetch_kddcup99(subset='SF', percent10=False)

In [4]:
df = pd.DataFrame(sf.data, columns=["duration", "service", "src_bytes", "dst_bytes"])
df.head()

,duration,service,src_bytes,dst_bytes
0,-2.302585,b'http',5.371103,10.716107
1,-2.302585,b'http',5.088213,8.418058
2,-2.302585,b'http',5.464255,7.113224
3,-2.302585,b'http',5.451468,7.616825
4,-2.302585,b'http',5.476882,6.186414


Coloana `service` (și array-ul `target`) conține șiruri binare. Nu vrem să ne dea bătăi de cap, deci le convertim în șiruri ASCII folosind [pandas.Series.str](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html). Asta ne va permite să folosim o interfață de [string](https://docs.python.org/3/library/string.html) pentru întreaga coloană.

In [5]:
df.service = df.service.str.decode('ascii')
target = pd.Series(sf.target).str.decode('ascii')

In [6]:
c = target.value_counts()
pd.DataFrame({'count': c, 'percentage': c / c.sum()}).style.format({'percentage': '{:.3%}'})

,count,percentage
normal.,699690,99.520%
back.,2203,0.313%
warezclient.,1020,0.145%
satan.,57,0.008%
buffer_overflow.,30,0.004%
ipsweep.,28,0.004%
loadmodule.,8,0.001%
ftp_write.,6,0.001%
rootkit.,5,0.001%
phf.,4,0.001%


In [7]:
c = df.service.value_counts()
pd.DataFrame({'count': c, 'percentage': c / c.sum()}).style.format({'percentage': '{:.3%}'})

,count,percentage
http,567498,80.717%
smtp,95156,13.534%
ftp_data,30464,4.333%
ftp,4091,0.582%
telnet,2141,0.305%
auth,1814,0.258%
pop_3,923,0.131%
other,434,0.062%
IRC,361,0.051%
X11,84,0.012%


In [8]:
services = [label for label, count in df.service.value_counts().items() if count >= 5000]
print(services)

X = df.drop(columns='service')
for l in services:
    X[f'service_{l}'] = df['service'] == l
        
X.head()

['http', 'smtp', 'ftp_data']


,duration,src_bytes,dst_bytes,service_http,service_smtp,service_ftp_data
0,-2.302585,5.371103,10.716107,True,False,False
1,-2.302585,5.088213,8.418058,True,False,False
2,-2.302585,5.464255,7.113224,True,False,False
3,-2.302585,5.451468,7.616825,True,False,False
4,-2.302585,5.476882,6.186414,True,False,False


In [9]:
y = target == 'normal.'
X_trn, X_val, y_trn, y_val = train_test_split(X, y, test_size=.25, stratify=y, random_state=0xBADA55)

X_trn.shape, X_val.shape, y_trn.shape, y_val.shape

((527300, 6), (175767, 6), (527300,), (175767,))

In [10]:
X_trn, X_val = X_trn.astype('float'), X_val.astype('float')
display(X_trn.describe())
display(X_val.describe())

,duration,src_bytes,dst_bytes,service_http,service_smtp,service_ftp_data
count,527300.000000,527300.000000,527300.000000,527300.000000,527300.000000,527300.000000
mean,-1.992965,5.849057,6.835322,0.807326,0.135215,0.043239
std,1.128464,0.911353,2.374900,0.394400,0.341954,0.203395
min,-2.302585,-2.302585,-2.302585,0.000000,0.000000,0.000000
25%,-2.302585,5.403128,5.869579,1.000000,0.000000,0.000000
50%,-2.302585,5.642262,7.089327,1.000000,0.000000,0.000000
75%,-2.302585,5.805436,8.077789,1.000000,0.000000,0.000000
max,10.609158,18.310660,16.277711,1.000000,1.000000,1.000000


,duration,src_bytes,dst_bytes,service_http,service_smtp,service_ftp_data
count,175767.000000,175767.000000,175767.000000,175767.000000,175767.000000,175767.000000
mean,-1.992207,5.848139,6.834883,0.806721,0.135731,0.043603
std,1.127054,0.912576,2.380513,0.394871,0.342503,0.204211
min,-2.302585,-2.302585,-2.302585,0.000000,0.000000,0.000000
25%,-2.302585,5.403128,5.855358,1.000000,0.000000,0.000000
50%,-2.302585,5.638710,7.091825,1.000000,0.000000,0.000000
75%,-2.302585,5.805436,8.081197,1.000000,0.000000,0.000000
max,10.328693,16.750908,14.888850,1.000000,1.000000,1.000000


## IsolationForest

Modelele de [anomaly detection](https://scikit-learn.org/stable/modules/outlier_detection.html) din scikit-learn au următoarele metode: 
* `score_samples`: Un scor de normalitate
* `decision_function`: Scorul de normalitate minus o constantă astfel încăt `N * contamination` din scoruri să fie negative
* `predict`: Semnul funcției `decision_function` (aparține {-1, 1})

<img src="data:image/png;base64, ..."/>

In [11]:
isofor = IsolationForest(n_estimators=25, max_samples=0.25, contamination='auto', n_jobs=-1)
%time isofor.fit(X_trn)

p_trn = isofor.score_samples(X_trn)
p_val = isofor.score_samples(X_val)

CPU times: user 2.31 s, sys: 245 ms, total: 2.55 s
Wall time: 637 ms


In [12]:
# %matplotlib widget
bins = np.linspace(min(p_trn.min(), p_val.min()), max(p_trn.max(), p_val.max()), 100)

plt.hist(p_trn, label='Train', histtype='step', bins=bins)
plt.hist(p_val, label='Validation', histtype='step', bins=bins)
plt.suptitle('IsolationForest sample score distribution')
plt.legend(loc='upper left');

<IPython.core.display.Javascript object>

Precizia pe o clasă reprezintă raportul de predicții corecte din toate cazurile când modelul prezice acea clasă
* Răspunde întrebării: Care sunt șansele să fi venit lupul, __având în vedere ca s-a strigat "Lupul"__?
* Poate fi definit și pentru clasa negativă: __Aseară a fost liniște__. Care sunt șansele ca oile să fie în regulă?

Senzitivitatea (recall / exhaustivitate) reprezintă raportul de predicții corecte în cadrul unei clase
* Răspunde întrebării: __În cazul unui atac__, care sunt șansele de a fi anunțat?
* Pentru clasa negativă: De câte ori voi fi trezit __degeaba, deci când nu au sosit de fapt lupii__?

Scorul F1 este media armonica a preciziei și a senzitivității și este foarte utila pentru a măsura performanța unui clasificator pe o problemă dezechilibrată/nebalansată.

In [13]:
# %matplotlib widget
my_contamination_offset = np.quantile(p_trn, .15)

print(classification_report(y_trn, p_trn > my_contamination_offset, target_names=['Anomaly', 'Normal']))
cm = confusion_matrix(y_trn, p_trn > my_contamination_offset)
sns.heatmap(cm, xticklabels=['Anomaly', 'Normal'], yticklabels=['Anomaly', 'Normal'], annot=True, fmt='d')

              precision    recall  f1-score   support

     Anomaly       0.03      1.00      0.06      2533
      Normal       1.00      0.85      0.92    524767

    accuracy                           0.85    527300
   macro avg       0.52      0.93      0.49    527300
weighted avg       1.00      0.85      0.92    527300



<IPython.core.display.Javascript object>

<AxesSubplot:>

In [14]:
# %matplotlib widget

# ROC curve vrea predicții în [0, 1], așa că vom transforma p_trn într-un clasament
p_trn_quant = 1 - pd.Series(p_trn).rank(pct=True)

precision, recall, t = precision_recall_curve(y_trn, p_trn_quant, pos_label=False)
plt.plot(1 - t, precision[:-1], label='Precision')
plt.plot(1 - t, recall[:-1], label='Recall')
plt.xlabel('Contamination ratio')
plt.grid()
plt.legend();

<IPython.core.display.Javascript object>

# Diabetes dataset

In [15]:
data = load_diabetes()
print(data['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

In [16]:
df = pd.DataFrame(data=data['data'], columns=data['feature_names'])

print(df.info())
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
dtypes: float64(10)
memory usage: 34.7 KB
None


,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
count,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02
mean,-3.639623e-16,1.309912e-16,-8.013951e-16,1.289818e-16,-9.042540e-17,1.301121e-16,-4.563971e-16,3.863174e-16,-3.848103e-16,-3.398488e-16
std,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02
min,-1.072256e-01,-4.464164e-02,-9.027530e-02,-1.123996e-01,-1.267807e-01,-1.156131e-01,-1.023071e-01,-7.639450e-02,-1.260974e-01,-1.377672e-01
25%,-3.729927e-02,-4.464164e-02,-3.422907e-02,-3.665645e-02,-3.424784e-02,-3.035840e-02,-3.511716e-02,-3.949338e-02,-3.324879e-02,-3.317903e-02
50%,5.383060e-03,-4.464164e-02,-7.283766e-03,-5.670611e-03,-4.320866e-03,-3.819065e-03,-6.584468e-03,-2.592262e-03,-1.947634e-03,-1.077698e-03
75%,3.807591e-02,5.068012e-02,3.124802e-02,3.564384e-02,2.835801e-02,2.984439e-02,2.931150e-02,3.430886e-02,3.243323e-02,2.791705e-02
max,1.107267e-01,5.068012e-02,1.705552e-01,1.320442e-01,1.539137e-01,1.987880e-01,1.811791e-01,1.852344e-01,1.335990e-01,1.356118e-01


In [17]:
# %matplotlib widget
sns.pairplot(df, hue='sex', vars=df.drop(columns='sex'), palette='coolwarm',
             plot_kws=dict(s=10, linewidth=0))

<IPython.core.display.Javascript object>

## LocalOutlierFactor

[Algoritmul LOF](http://www.cse.ust.hk/~leichen/courses/comp5331/lectures/LOF_Example.pdf)

TL;DR: Densitatea populației pe glob variază foarte mult de la țară la țară, deci probabil și numărul mediu de prieteni ai unui om și nivelul de interacțiune dintre oameni. Dacă cei mai apropiați 5/10/100 de prieteni ai tăi nu te consideră unul din cei mai apropiați prieteni ai lor, indică faptul că ești un caz anormal de izolat.

In [18]:
trn, val = train_test_split(df, test_size=.25, random_state=0xBADA55)

lof = LocalOutlierFactor(n_neighbors=10, novelty=True, contamination='auto',
                         metric='euclidean', n_jobs=-1)
%time lof.fit(trn.to_numpy())

p_trn = lof.negative_outlier_factor_
p_val = lof.score_samples(val)

CPU times: user 3.63 ms, sys: 1.77 ms, total: 5.4 ms
Wall time: 5.14 ms


In [19]:
bins = np.linspace(min(p_trn.min(), p_val.min()), max(p_trn.max(), p_val.max()), 50)

plt.hist(p_trn, label='Train', histtype='step', bins=bins)
plt.hist(p_val, label='Validation', histtype='step', bins=bins)
plt.suptitle('LocalOutlierFactor sample score distribution')
plt.legend(loc='upper left');

<IPython.core.display.Javascript object>

In [20]:
# Implementation of a P-P plot which might indicate overfitting

plt.plot([0, 1], [0, 1], 'k:')

for p in p_trn:
    a = np.mean(p_trn <= p)
    b = np.mean(p_val <= p)
    plt.scatter(a, b, c='tab:blue', s=5)
    
plt.xlabel('Theoretical (Training) distribution')
plt.ylabel('Observed (Validation) distribution')
plt.xlim([0, 1]), plt.ylim([0, 1]);

<IPython.core.display.Javascript object>

https://towardsdatascience.com/explaining-probability-plots-9e5c5d304703

In [21]:
data = pd.concat([trn.assign(anomaly=p_trn, is_trn=True), val.assign(anomaly=p_val, is_trn=False)])
data.anomaly = data.anomaly.rank()

pg = sns.PairGrid(data, hue='anomaly', vars=df.drop(columns='sex').columns[:5], palette='Spectral')
pg.data = data[data.is_trn]
pg.map_lower(plt.scatter, s=15, lw=.25)
pg.data = data[~data.is_trn]
pg.map_upper(plt.scatter, s=15, lw=.25)

<IPython.core.display.Javascript object>